
# Week 7 – Control Systems: PID & State-Space Models ⚙️

In this week, we'll learn the basics of control systems relevant to aerospace applications:
- PID controllers (design and tuning)
- Simulation of closed-loop systems
- State-space representation and analysis

---
### ✅ Learning Goals
- Understand PID controller components (P, I, D)
- Implement a PID controller in Python
- Simulate a simple aircraft pitch/altitude control loop
- Represent systems in state-space form and compute responses


In [ ]:

# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.integrate import solve_ivp

# Note: For advanced control features, install the 'control' package:
# pip install control
try:
    import control
    has_control = True
except Exception as e:
    has_control = False
    print("control library not available; examples will use scipy.signal as fallback.")



## 🔧 PID Controller Overview
PID has three terms:
- **P (Proportional):** reacts to current error
- **I (Integral):** reacts to accumulated error (eliminates steady-state error)
- **D (Derivative):** reacts to rate of change of error (damps overshoot)

Control law:
\[ u(t) = K_p e(t) + K_i \int e(t) dt + K_d \frac{d e(t)}{dt} \]


In [ ]:

# Simple PID controller implementation
class PIDController:
    def __init__(self, Kp=1.0, Ki=0.0, Kd=0.0, setpoint=0.0, dt=0.01, output_limits=(None,None)):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.setpoint = setpoint
        self.dt = dt
        self.output_limits = output_limits
        self._integral = 0.0
        self._prev_error = 0.0

    def reset(self):
        self._integral = 0.0
        self._prev_error = 0.0

    def update(self, measurement):
        error = self.setpoint - measurement
        self._integral += error * self.dt
        derivative = (error - self._prev_error) / self.dt if self.dt > 0 else 0.0
        self._prev_error = error

        u = self.Kp*error + self.Ki*self._integral + self.Kd*derivative

        # Apply output limits
        low, high = self.output_limits
        if low is not None:
            u = max(low, u)
        if high is not None:
            u = min(high, u)
        return u



## 🧪 Example: PID controlling a first-order plant (altitude hold simplified)

Plant: \( \dot{x} = -a x + b u \) where x is altitude error, u is control input (engine/thrust effect).  
We'll simulate a closed-loop system using the PID controller class.


In [ ]:

# Plant parameters (simple linear model)
a = 0.05   # natural decay (disturbance damping)
b = 0.1    # control effectiveness

# Simulation parameters
dt = 0.1
T = 60.0
time = np.arange(0, T, dt)

# Desired altitude (setpoint)
setpoint = 1000.0  # meters

# Initial altitude (measurement)
altitude = 900.0

# Create PID controller
pid = PIDController(Kp=0.8, Ki=0.02, Kd=0.1, setpoint=setpoint, dt=dt, output_limits=(0, 1000))

# Logging
alt_history = []
u_history = []

for t in time:
    u = pid.update(altitude)
    # Plant update: dx/dt = -a*(altitude - ambient) + b*u
    # Use simple Euler integration for demonstration
    d_alt = -a*(altitude - 0) + b*u
    altitude = altitude + d_alt*dt
    alt_history.append(altitude)
    u_history.append(u)

# Plot results
plt.figure(figsize=(10,5))
plt.plot(time, alt_history, label='Altitude (m)')
plt.axhline(setpoint, color='k', linestyle='--', label='Setpoint')
plt.xlabel('Time (s)')
plt.ylabel('Altitude (m)')
plt.title('PID Altitude Hold Simulation (Simple Plant)')
plt.legend()
plt.grid(True)

plt.figure(figsize=(10,3))
plt.plot(time, u_history, label='Control Input (u)')
plt.xlabel('Time (s)')
plt.ylabel('Control Effort')
plt.title('Control Input over Time')
plt.grid(True)
plt.show()



## 📐 State-Space Representation

State-space form:
\[ \dot{x} = A x + B u \]
\[ y = C x + D u \]

We will build a simple second-order system (e.g., pitch dynamics approximation) and compute its response.


In [ ]:

# Second-order system parameters (example)
wn = 1.0   # natural frequency
zeta = 0.5 # damping ratio

# Transfer function: G(s) = wn^2 / (s^2 + 2*zeta*wn*s + wn^2)
num = [wn**2]
den = [1, 2*zeta*wn, wn**2]

# Convert to state-space using scipy.signal
sys_tf = signal.TransferFunction(num, den)
sys_ss = sys_tf.to_ss()

A = sys_ss.A
B = sys_ss.B
C = sys_ss.C
D = sys_ss.D

A, B, C, D


In [ ]:

# Time for simulation
t = np.linspace(0, 20, 500)

# Step response using scipy.signal
t_out, y_out = signal.step(signal.TransferFunction(num, den), T=t)

plt.figure(figsize=(8,4))
plt.plot(t_out, y_out)
plt.xlabel('Time (s)')
plt.ylabel('Response')
plt.title('Step Response of Second-Order System')
plt.grid(True)
plt.show()



## 🔁 (Optional) Using `control` library for classical design

If the `control` library is available, you can more easily design controllers, compute Bode plots, root loci, and analyze state-space systems. Example code (run only if `control` is installed):


In [ ]:

if has_control:
    # PID using control library
    G = control.TransferFunction(num, den)
    Kp = 1.0
    Ki = 0.1
    Kd = 0.05
    C = control.pid(Kp, Ki, Kd)  # uses control library's PID helper (if available)
    closed_loop = control.feedback(C*G, 1)
    t_cl, y_cl = control.step_response(closed_loop, T=np.linspace(0, 20, 500))
    import matplotlib.pyplot as plt
    plt.figure(figsize=(8,4))
    plt.plot(t_cl, y_cl)
    plt.title('Closed-loop Step Response with PID (control library)')
    plt.grid(True)
    plt.show()
else:
    print('control library not installed; skip advanced example.')



---
### ✅ Summary
- Implemented a PID controller and simulated a simple closed-loop altitude hold.  
- Explored state-space representation and step response for a second-order system.  
- Recommended next steps: tune PID gains (Ziegler–Nichols, trial-and-error), analyze frequency response (Bode), and design state feedback controllers (LQR) in advanced modules.
